In [ ]:
! git clone https://github.com/mmarouen/Road-Lane-Semantic-segmentation.git
%cd Road-Lane-Semantic-segmentation/

In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/Road-Lane-Semantic-segmentation"
!chmod 755 load_from_gdrive.sh

In [ ]:
!python roadseg/scripts/train_chain.py -c configs.json

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
target_file = "/content/gdrive/My Drive/semantic_segmentation/fcn32.h5"

In [ ]:
!mkdir -p "/content/gdrive/My Drive/semantic_segmentation/plots"
!cp plots/training_perf.png "/content/gdrive/My Drive/semantic_segmentation/plots/training_perf.png"
!cp models/fcn32.h5 "/content/gdrive/My Drive/semantic_segmentation/fcn32.h5"

In [14]:
!pip install PyDrive #get files from gdrive

In [15]:
import os.path
import tensorflow as tf
import warnings
from distutils.version import LooseVersion
from google.colab import files
import shutil
import keras

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
# check keras version
print("Keras version: "+keras.__version__)

Keras version: 2.4.3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: No GPU found. Please use a GPU to train your neural network.
  # This is added back by InteractiveShellApp.init_path()


In [16]:
from google.colab import files
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
import time

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [17]:
weights=drive.CreateFile({'id':'1uyOrTF7oxL2dyFQh9iQ81hIzPJlEl2KH'})
weights.GetContentFile('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [18]:
dataset=drive.CreateFile({'id':'1v4CQv1Q3eag8xRwG1cgbcD-CXjzJHmXu'})
dataset.GetContentFile('data_road.zip')
zip_ref = zipfile.ZipFile("data_road.zip",'r')
zip_ref.extractall()
zip_ref.close()

In [19]:
utils_file=drive.CreateFile({'id':'10YDp9U3I3r2XAMJD3IGedbthY0aZZ4de'})
utils_file.GetContentFile('ss_utils.py')
import ss_utils

In [20]:
#fcn32=drive.CreateFile({'id':'19pCbgniAOIILJ7nlAzfDxQNtu5gTQx_Y'})
#fcn32.GetContentFile('fcn32.h5')

ApiRequestError: ignored

In [21]:
#fcn8=drive.CreateFile({'id':'1_IuGHgD7_4hwmaw-KxV63rsOLS03WDnx'})
#fcn8.GetContentFile('fcn8.h5')

ApiRequestError: ignored

In [22]:
import os
import numpy as np
import cv2
import itertools
from glob import glob
import re
import random
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import *
from keras.layers import *
from keras import models
from keras import layers
from keras.utils import plot_model
from keras import optimizers
from keras.models import load_model
import matplotlib.pyplot as plt
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
from keras import backend as K
from glob import glob

In [24]:
base_dir='data_road'
train_dir = os.path.join(base_dir, 'training')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'testing')
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
IMAGE_ORDERING = 'channels_first'
InputHeight=160
InputWidth=576
img_input = Input(shape=(InputHeight,InputWidth,3))
nClasses=2
drive_service = build('drive', 'v3')

In [25]:
train_images,train_labels=ss_utils.get_data(train_dir, nClasses ,InputHeight, InputWidth)
val_images,val_labels=ss_utils.get_data(val_dir, nClasses, InputHeight, InputWidth)
train_Aug_lab,train_Aug_im=ss_utils.augment_training_data(train_images,train_labels,nClasses,InputHeight,InputWidth)
print('training samples: '+str(len(train_images)))
print('validation samples: '+str(len(val_images)))
print('training samples after augmentation: '+str(len(train_Aug_lab)))

training samples: 255
validation samples: 34
training samples after augmentation: 510


In [26]:
fcn32model=ss_utils.fcn32(nClasses,'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',InputHeight,InputWidth)
fcn32model.summary()
fcn32model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=1e-4),metrics=['acc'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 160, 576, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 160, 576, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 160, 576, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 80, 288, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 80, 288, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 80, 288, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 40, 144, 128)      0   

In [27]:
history = fcn32model.fit(x=train_Aug_im, y=train_Aug_lab,
                    epochs=20,
                    batch_size=10,
                    validation_data=(val_images, val_labels))

Epoch 1/20
51/51 [==============================] - 896s 18s/step - loss: 0.6634 - acc: 0.5903 - val_loss: 0.4373 - val_acc: 0.8425
Epoch 2/20
51/51 [==============================] - 900s 18s/step - loss: 0.3625 - acc: 0.8731 - val_loss: 0.2255 - val_acc: 0.9136
Epoch 3/20
24/51 [=============>................] - ETA: 7:33 - loss: 0.2140 - acc: 0.9175

KeyboardInterrupt: ignored

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:

fcn32model.save('fcn32.h5')
ss_utils.save_file_to_drive(drive_service,"fcn32.h5", "fcn32.h5")

In [ ]:
fcn32= load_model('fcn32.h5')

In [ ]:
fcn8model=ss_utils.fcn8(fcn32model,nClasses)
fcn8model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=1e-4),metrics=['acc'])
plot_model(fcn8model)
fcn8model.summary()

In [ ]:
fcn8model=load_model('fcn8.h5')

In [ ]:
# K.set_value(fcn8model.optimizer.lr, 0.00001)

In [ ]:
history = fcn8model.fit(x=train_Aug_im, y=train_Aug_lab,
                    epochs=20,
                    batch_size=10,
                    validation_data=(val_images, val_labels))

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
fcn8model.save('fcn8.h5')
ss_utils.save_file_to_drive(drive_service,"fcn8.h5", "fcn8.h5")

In [ ]:
fcn8model=load_model('fcn8.h5')

In [ ]:
fcn32model=load_model('fcn32.h5')

In [ ]:
%matplotlib inline
plt.rcParams["axes.grid"] = False

image_paths = glob(os.path.join('data_road/training/', 'image_2', '*.png'))
nImages=3
idx=np.random.choice(len(image_paths),size=nImages)
for i,id in enumerate(idx):
  imgName=image_paths[id]
  outName = os.path.join('predictions',os.path.basename(imgName))
  im0 = cv2.imread(imgName, 1)
  im,seg_img=ss_utils.fcn_predict(fcn8model,im0,nClasses,InputHeight,InputWidth)
  fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
  if i==0: plt.subplot(1, 2, 1).set_title('FCN8')
  else: plt.subplot(1, 2, 1)
  plt.imshow(cv2.addWeighted(im,0.9,seg_img,0.5,0))
  if i==0: plt.subplot(1, 2, 2).set_title('FCN32')
  else: plt.subplot(1, 2, 2)
  im,seg_img2=ss_utils.fcn_predict(fcn32model,im0,nClasses,InputHeight,InputWidth)
  plt.imshow(cv2.addWeighted(im,0.9,seg_img2,0.5,0))
  if i==(len(idx)-1): plt.show()
  else: plt.figure()